In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [49]:
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [5]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [6]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [2]:
import re
import string

#Cleanup the strings so we only get important words

#Remove any urls
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

#Remove any extra html
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

#Remove any emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Remove puncations and hashtags
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def clean_tweet(text):
    text = remove_url(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = remove_punct(text)
    return text

#Only apply this to train_df
train_df['text'] = train_df['text'].apply(lambda x : clean_tweet(x))
test_df['text'] = test_df['text'].apply(lambda x : clean_tweet(x))

In [65]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1
5,8,NaN,NaN,RockyFire Update California Hwy 20 closed in ...,1
6,10,NaN,NaN,flood disaster Heavy rain causes flash floodin...,1
7,13,NaN,NaN,Im on top of the hill and I can see a fire in ...,1
8,14,NaN,NaN,Theres an emergency evacuation happening now i...,1
9,15,NaN,NaN,Im afraid that the tornado is coming to our area,1


In [3]:
# Split the data into train and validation sets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert the text into vectors
vectorizer = CountVectorizer()

x_train, x_test, y_train, y_test = train_test_split(train_df['text'], train_df['target'], test_size=0.3, random_state=42)

#After splitting into train and test, tokenize
x_train_vectors = vectorizer.fit_transform(x_train)
x_test_vectors = vectorizer.transform(x_test)

In [81]:
x_train_vectors

<5329x14210 sparse matrix of type '<class 'numpy.int64'>'
	with 68468 stored elements in Compressed Sparse Row format>

In [82]:
y_train

1186    0
4071    1
5461    1
5787    1
7445    0
       ..
5226    0
5390    0
860     0
7603    1
7270    1
Name: target, Length: 5329, dtype: int64

In [5]:
from sklearn.neighbors import KNeighborsClassifier

#Just a basic kneighbors to get something working

clf = KNeighborsClassifier(n_neighbors=3)

clf.fit(x_train_vectors, y_train);

In [6]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_test_vectors)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy is {accuracy*100:.2f}%")

Accuracy is 69.35%


In [8]:
sample_submission = pd.read_csv("sample_submission.csv")

In [11]:
sample_submission["target"] = clf.predict(x_test_vectors)

ValueError: Length of values (2284) does not match length of index (3263)

In [12]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [13]:
sample_submission.to_csv("submission.csv", index=False)

In [21]:
from sklearn.naive_bayes import MultinomialNB

x_train = train_df['text']
y_train  = train_df['target']
x_test = test_df['text']

model = MultinomialNB(alpha=1.1)
x_train_vectors = vectorizer.fit_transform(x_train)
x_test_vectors = vectorizer.transform(x_test)
model.fit(x_train_vectors, y_train)

predicted = model.predict(x_test_vectors)
makeSubmissionFile(predicted)

In [16]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = predicted

In [5]:
def makeSubmissionFile(y_pred):
    fh = open("submission.csv", "w")
    fh.write("id,target\n")
    for i in range (len(y_pred)):
        pid = test_df['id'][i]
        fh.write(str(pid))
        fh.write(',')
        fh.write(str(y_pred[i]))
        fh.write('\n')
    fh.close()

In [22]:
makeSubmissionFile(predicted)

In [17]:
from sklearn.model_selection import GridSearchCV

param_grid_nb = {
    'alpha': np.logspace(0,5, num=100),
    'fit_prior' : (True, False)
}
model_grid = GridSearchCV(estimator=MultinomialNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
model_grid.fit(x_train_vectors, y_train)
print(model_grid.best_estimator_)
predicted = model_grid.predict(x_test_vectors)
makeSubmissionFile(predicted)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s


MultinomialNB(alpha=12.91549665014884)


[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    1.3s finished


In [13]:
print(model_grid.best_params_)
print(model_grid.best_score_)

{'alpha': 12.91549665014884, 'fit_prior': True}
0.7287570574703129


In [15]:
from sklearn.ensemble import AdaBoostClassifier
param_grid_nb = {
    'n_estimators': [10, 50, 100, 500],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1.0]
}
abc =AdaBoostClassifier(base_estimator=MultinomialNB())
model = abc.fit(x_train_vectors, y_train)
model_grid = GridSearchCV(estimator=abc, param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1,scoring='accuracy')
model_grid.fit(x_train_vectors, y_train)
print(model_grid.best_params_)
print(model_grid.best_score_)


Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:    4.6s
/Users/madhumithav/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   29.8s finished


{'learning_rate': 1.0, 'n_estimators': 100}
0.7178472516822387


In [8]:
predicted = model_grid.predict(x_test_vectors)
makeSubmissionFile(predicted)